**Libreria <a href="https://pandas.pydata.org/docs/getting_started/index.html"> Pandas**</a>

*Permite el procesamiento de datos sobre dataframes*

In [1]:
import pandas as pd

**Libreria  <a href="https://docs.python.org/3.12/library/ast.html"> Abstract Syntax Trees**</a>

*Permite la lectura literal de la linea*

In [2]:
import ast

# **Extraction**  

In [3]:
list = []
for line in open('G:\\My Drive\\IT\\dataset\\australian_user_reviews.json', encoding='utf-8').readlines(): # australian_user_reviews.json
    list.append(ast.literal_eval(line))
df_australian_User_Reviews = pd.DataFrame.from_dict(list)

In [4]:
list = []
for line in open('G:\\My Drive\\IT\\dataset\\australian_users_items.json', encoding='utf-8').readlines(): # australian_users_items.json
    list.append(ast.literal_eval(line))
df_australian_User_Items = pd.DataFrame.from_dict(list)

In [5]:
df_output_Steam_Games = pd.read_json('G:\\My Drive\\IT\\dataset\\output_steam_games.json', lines=True) #output_steam_games.json

# **Transform**

**`df_User_Reviews`**

In [17]:
#Creo df_User_Reviews'
df_Reviews = df_australian_User_Reviews[['user_id','reviews']]

In [18]:
#Desanido los valores de 'reviews'
df_Reviews = df_Reviews.explode('reviews')

In [19]:
#Reseteo el index 
df_Reviews.reset_index(drop=True, inplace=True)

In [20]:
#Normalizo la columna 'reviews'
reviews_normalizado = pd.json_normalize(df_Reviews['reviews'])

In [21]:
#Concateno la columna 'reviews' normalizada, con el df original.
df_Reviews = pd.concat([df_Reviews.drop(columns=['reviews']), reviews_normalizado], axis=1)

In [22]:
#Elimino columnas 'last_edited', 'funny', 'helpful'
df_Reviews = df_Reviews[['user_id','posted','item_id','recommend','review']]

In [23]:
#Modifico el valor de la clave 'posted'    
df_Reviews['posted'] = df_Reviews['posted'].replace("Posted ","", regex=True) #Reemplazo 'Posted' --> ''
df_Reviews['posted'] = df_Reviews['posted'].str.rstrip('.') #Eliminio '.' --> ''

*Limpio las fechas que estan incompletas por la falta del año del posteo*

In [24]:
#Seteo el tipo de dato de la columna 'posted'
df_Reviews['posted'] = df_Reviews['posted'].apply(str)

In [25]:
#Creo una funcion para corroborar que la fecha cumpla con: cantidad de caracteres de una fecha valida >12 y la presencia del separador ','
def fecha_Valida(fecha):
    return len(fecha) > 12 or ',' in fecha

In [26]:
#Implemento la funcion sobre la columna 'posted'
df_Reviews['posted'] = df_Reviews['posted'][df_Reviews['posted'].apply(fecha_Valida)] #Reutilizo df resultante como mascara booleana para eliminr las fechas incompletas

In [27]:
#Elimino NaNs 
df_Reviews.dropna(inplace=True)

*Importo `datetime`, para cambiar el formato de los valores de 'Posted' a '%d-%m-%Y'* 

In [28]:
#Seteo el tipo de dato de la columna 'posted' a datetime
df_Reviews['posted'] = pd.to_datetime(df_Reviews['posted']) 

In [29]:
#Seteo el tipo de dato de la columna 'item_id' a int
df_Reviews['item_id'] = df_Reviews['item_id'].apply(int)

In [30]:
#Seteo el tipo de dato de la columna 'recomend' a int
df_Reviews['recommend'] = df_Reviews['recommend'].apply(int)

**Feature Engineering: Sentiments Analysis con <a href="https://www.nltk.org/"> NLTK </a>** 
 

*Descargo<a href="https://www.nltk.org/nltk_data/"> `VADER Sentiment Lexicon` </a>* 

*Contiene un conjunto de palabras con sus respectivos puntajes de polaridad, lo que ayuda a determinar la intensidad del sentimiento expresado en una frase.*

In [31]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Error loading vader_lexicon: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [32]:
# Importo clase SentimentIntensityAnalyzer   
from nltk.sentiment.vader import SentimentIntensityAnalyzer 

In [33]:
# Instancio SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer() 

In [34]:
# Implemento funcion 'polarity_scores' sobre df_User_Reviews
df_Reviews['sentiment_analysis'] = df_Reviews['review'].apply(sia.polarity_scores)
df_Reviews['sentiment_analysis'] = df_Reviews['sentiment_analysis'].apply(lambda row: row['compound']) #Espicifico la clave 'compound', porque resume la intesidad del sentimiento expresado en la frase 

In [35]:
# Recategorizo la intensidad del sentimiento expresado en cada frase
""" Nueva Escala
        
        POSITIVO -> 2
        NEUTRO -> 1
        NEGATIVO -> 0

 """ 

def recategorizar(int):
    if(int>0):
        return 2
    else:
        if(abs(int) == 0):
            return 1
        else:
            return 0 
        
df_Reviews['sentiment_analysis'] = df_Reviews['sentiment_analysis'].apply(recategorizar)

**`df_Users_Items`**

In [57]:
#Creo df_Users_Items'
df_Items = df_australian_User_Items[['user_id','items_count','items']]

In [58]:
#Desanido 'items'
df_Items=df_Items.explode('items')

In [59]:
#Reseteo Index
df_Items.reset_index(drop=True, inplace=True)

In [60]:
#Normalizo la columna 'items'
items_normalizado = pd.json_normalize(df_Items['items'])

In [61]:
#Concateno la columna 'items' normalizada, con el df original.
df_Items = pd.concat([df_Items.drop(columns=['items']), items_normalizado], axis=1)

In [62]:
#Elimino nulos
df_Items.dropna(inplace=True) 

In [63]:
#Elimino columna 'playtime_2weeks', 'item_name', porque no son datos relevantes en el desarrollo del trabajo y reduce el tamaño del df. 
df_Items = df_Items[['user_id','items_count','item_id','playtime_forever']]

In [64]:
#Seteo el tipo de dato de la columna 'item_id' a int
#Motivo, conociendo los tipos de valores de clave es mas sencillo trabajar con int, antes que str.
df_Items['item_id'] = df_Items['item_id'].apply(int)

In [68]:
df_Items['playtime_forever'] = df_Items['playtime_forever'].apply(int)

In [75]:
df_Reviews

,user_id,posted,item_id,recommend,review,sentiment_analysis
0,76561197970982479,2011-11-05,1250,1,Simple yet with great replayability. In my opi...,2
1,76561197970982479,2011-07-15,22200,1,It's unique and worth a playthrough.,2
2,76561197970982479,2011-04-21,43110,1,Great atmosphere. The gunplay can be a bit chu...,2
3,js41637,2014-06-24,251610,1,I know what you think when you see this title ...,2
4,js41637,2013-09-08,227300,1,For a simple (it's actually not all that simpl...,2
...,...,...,...,...,...,...
59280,wayfeng,2015-10-14,730,1,its FUNNNNNNNN,1
59283,76561198251004808,2015-10-10,253980,1,Awesome fantasy game if you don't mind the gra...,2
59293,72947282842,2015-10-31,730,1,Prettyy Mad Game,0
59295,ApxLGhost,2015-12-14,730,1,AMAZING GAME 10/10,2


In [69]:
df_Items

,user_id,items_count,item_id,playtime_forever
0,76561197970982479,277,10,6
1,76561197970982479,277,20,0
2,76561197970982479,277,30,7
3,76561197970982479,277,40,0
4,76561197970982479,277,50,0
...,...,...,...,...
5170009,76561198329548331,7,346330,0
5170010,76561198329548331,7,373330,0
5170011,76561198329548331,7,388490,3
5170012,76561198329548331,7,521570,4


**`df_Steam_Games`**

In [50]:
#Creo df unicamente con los siguientes campos: {genres, app_name, id, app_name}
#Porque el resto de la columna no aportan datos relevantes para la implementacion de las funciones con la API.
df_Games = df_output_Steam_Games[['id','genres']]

In [51]:
#Elimino nulos 
df_Games.dropna(inplace=True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_12948\1541470593.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Games.dropna(inplace=True)


In [52]:
#Seteo el tipo de dato de la comlumna 'id' a int
df_Games['id'] = df_Games['id'].apply(int)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_12948\1246554794.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Games['id'] = df_Games['id'].apply(int)


In [53]:
df_Games = df_Games.explode('genres')

In [54]:
#Reset index
df_Games.reset_index(drop=True, inplace=True)

In [72]:
df_Games['genres'] = df_Games['genres'].apply(str.lower)

In [73]:
df_Games

,id,genres
0,761140,action
1,761140,casual
2,761140,indie
3,761140,simulation
4,761140,strategy
...,...,...
71547,610660,indie
71548,610660,racing
71549,610660,simulation
71550,658870,casual


**Sección Games_Id**

In [40]:
#Creo df exclusivo para con el nombre e id del juego
df_Games_Id = df_output_Steam_Games[['id','app_name']]

In [41]:
df_Games_Id.dropna(inplace=True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_12948\3022317277.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Games_Id.dropna(inplace=True)


In [42]:
#Seteo el tipo de dato de la comlumna 'id' a int
df_Games_Id['id'] = df_Games_Id['id'].apply(int)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_12948\2675047912.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Games_Id['id'] = df_Games_Id['id'].apply(int)


In [43]:
df_Games_Id.reset_index(drop=True, inplace=True)

# **Load**

*Cargo los dataframes procesados en formato .parquet por el tamaño del archivo resultante que devuelve este tipo de compresion.*

**`df_User_Reviews`**

In [37]:
df_Reviews.to_parquet('user_Reviews')

**`df_Users_Items`**

In [70]:
df_Items.to_parquet('user_Items')

**`df_Steam_Games`**

In [74]:
df_Games.to_parquet('steam_Games')